# RDD chiave-valore
In questo notebook vedremo come possiamo creare un Resilient Distributed Dataset (RDD) usando una rappresentazione del contenuto chiave-valore. Cominciamo inizializzando la sessione.

In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("basic")
sc = SparkContext(conf=conf)

Creiamo un RDD di esempio, contenente degli acquisti effettuati all'interno di un app, ogni elemento sarà caratterizzato da una lista contenente:
* **Username**: che fungerà da chiave.
* **Item**: una nuova lista che contiene id dell'item acquistato ed il prezzo.

Quando utilizziamo una rappresentazione a lista, il primo elemento viene sempre interpretato da spark come la chiave ed il secondo come il valore.

In [20]:
purchases = [("guizard", ("pacchetto-crediti-1", "0.89 €")),
       ("bitleader", ("pacchetto-crediti-1", "0.89 €")),
       ("guizard",  ("ads-remover", "4.99 €")),
       ("guizard", ("pacchetto-crediti-3", "1.99 €")),
       ("bitleader", ("potenziamento-1", "1.49 €")),
       ("bitleader", ("potenziamento-2", "2.99 €")),
       ("lightlord", ("ads-remover", "4.99 €")),
       ("peanut", ("pacchett-crediti-1", "0.89 €")),
       ("lightlord", ("pacchetto-crediti-3", "4.99 €"))]

purchasesRDD = sc.parallelize(purchases)
purchasesRDD.collect()

[('guizard', ('pacchetto-crediti-1', '0.89 €')),
 ('bitleader', ('pacchetto-crediti-1', '0.89 €')),
 ('guizard', ('ads-remover', '4.99 €')),
 ('guizard', ('pacchetto-crediti-3', '1.99 €')),
 ('bitleader', ('potenziamento-1', '1.49 €')),
 ('bitleader', ('potenziamento-2', '2.99 €')),
 ('lightlord', ('ads-remover', '4.99 €')),
 ('peanut', ('pacchett-crediti-1', '0.89 €')),
 ('lightlord', ('pacchetto-crediti-3', '4.99 €'))]

## Map e Reduce con chiave
Per eseguire una trasformazione al contenuto del RDD, ma non alle chiavi, possiamo utilizzare il metodo *.mapValues(func)*.
<br>
Ad esempio convertiamo in maiuscolo l'item id. 

In [21]:
purchasesRDD = purchasesRDD.mapValues(lambda x: (x[0].upper(), x[1]))
purchasesRDD.collect()

[('guizard', ('PACCHETTO-CREDITI-1', '0.89 €')),
 ('bitleader', ('PACCHETTO-CREDITI-1', '0.89 €')),
 ('guizard', ('ADS-REMOVER', '4.99 €')),
 ('guizard', ('PACCHETTO-CREDITI-3', '1.99 €')),
 ('bitleader', ('POTENZIAMENTO-1', '1.49 €')),
 ('bitleader', ('POTENZIAMENTO-2', '2.99 €')),
 ('lightlord', ('ADS-REMOVER', '4.99 €')),
 ('peanut', ('PACCHETT-CREDITI-1', '0.89 €')),
 ('lightlord', ('PACCHETTO-CREDITI-3', '4.99 €'))]

E rimuoviamo il simbolo di EURO dal prezzo.

In [22]:
purchasesRDD = purchasesRDD.mapValues(lambda x: (x[0], float(x[1].split(" €")[0])))
purchasesRDD.collect()

[('guizard', ('PACCHETTO-CREDITI-1', 0.89)),
 ('bitleader', ('PACCHETTO-CREDITI-1', 0.89)),
 ('guizard', ('ADS-REMOVER', 4.99)),
 ('guizard', ('PACCHETTO-CREDITI-3', 1.99)),
 ('bitleader', ('POTENZIAMENTO-1', 1.49)),
 ('bitleader', ('POTENZIAMENTO-2', 2.99)),
 ('lightlord', ('ADS-REMOVER', 4.99)),
 ('peanut', ('PACCHETT-CREDITI-1', 0.89)),
 ('lightlord', ('PACCHETTO-CREDITI-3', 4.99))]

**NOTA BENE** Avremmo anche potuto eseguire le due operazioni in simultanea.

In [23]:
# purchasesRDD = purchasesRDD.mapValues(lambda x: (x[0].upper(), float(x[1].split(" €")[0])))

Un'altra operazione comune quando si lavora con una struttura dati in formato chiave-valore è il voler raggruppare i dati in base alla chiave, con un RDD possiamo farlo usando il metodo *.reduceByKey(func)*, usiamolo per sommare gli acquisti effettuati da ogni utente ed ottenere il valore totale. Per prima cosa creiamo una nuovo RDD che contiene soltanto il nome utente come chiave ed il costo dell'acquisto come valore.

In [28]:
totalByUserRDD = purchasesRDD.mapValues(lambda x: x[1])
totalByUserRDD.collect()

[('guizard', 0.89),
 ('bitleader', 0.89),
 ('guizard', 4.99),
 ('guizard', 1.99),
 ('bitleader', 1.49),
 ('bitleader', 2.99),
 ('lightlord', 4.99),
 ('peanut', 0.89),
 ('lightlord', 4.99)]

Poi usiamo il metodo *reduceByKey* per sommare i valori ed ottenere la spesa totale dell'utente all'interno dell'app.

In [29]:
totalByUserRDD = totalByUserRDD.reduceByKey(lambda x,y: x+y)
totalByUserRDD.collect()

[('guizard', 7.87), ('bitleader', 5.37), ('lightlord', 9.98), ('peanut', 0.89)]

E se volessimo sapere l'entrate totali delle app ? Dovremmo sommare la spesa di tutti gli utenti

In [48]:
total = totalByUserRDD.map(lambda x: x[1]).reduce(lambda x,y: x+y)
total

24.11

## Ordinamento di un RDD
Il metodo *.sortBy(func)* ci permette di ordinare gli elementi di un RDD in base ad una delle sue proprietà o ad una funzione specificata da noi. Questo metodo non richiede di avere i dati in formato chiave valore, quindi è sempre utilizzabile. 
<br>
Utilizziamolo per ordinare l'RDD in base all'importo speso da ogni utente.

In [35]:
totalByUserSortedRDD = totalByUserRDD.sortBy(lambda x: x[1])
totalByUserSortedRDD.collect()

[('peanut', 0.89), ('bitleader', 5.37), ('guizard', 7.87), ('lightlord', 9.98)]

Come vedi la lista va dal minore (0.89) al maggiore (9.98), possiamo invertire l'ordine di ordinamento impostando il parametro *ascending* a False.

In [34]:
totalByUserSortedRDD = totalByUserRDD.sortBy(lambda x: x[1], ascending=False)
totalByUserSortedRDD.collect()

[('lightlord', 9.98), ('guizard', 7.87), ('bitleader', 5.37), ('peanut', 0.89)]